In [ ]:
import os
import sys
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import skimage.io as io
from src.data_manager import DataManager
import src.watershed_extraction as we
import notebooks.plotutils as pltutils

GEN_PATH="generated" #path to save generated images
EXPORT_DPI=300

In [ ]:
data_manager = DataManager("assets") #local (relative to cwd) assets directory

print("Select index from:")
images = data_manager.get_input_images()
for i, file in enumerate(images):
    print("{}: {}".format(file, i))

   

In [ ]:
choice=21
image = io.imread(images[choice])
steps = we.detect_cells(image,return_steps=True)

### First Step: Image Smoothing

In [ ]:
fig_step_1, fig_1_axs = plt.subplots(ncols=2,figsize=(10,5), dpi=300)

fig_1_axs[0].imshow(steps.input)
fig_1_axs[0].set_title("Input image")
fig_1_axs[0].set_axis_off()

fig_1_axs[1].imshow(steps.meanshift)
fig_1_axs[1].set_title("Mean-shifted image")
fig_1_axs[1].set_axis_off()

fig_step_1.savefig(os.path.join(GEN_PATH,"step#1.png"),transparent=True, frameon=False)

### Second Step:  Thresholding Binarization

In [ ]:
fig_step_2, fig_2_axs = plt.subplots(ncols=3,figsize=(15,5), dpi=EXPORT_DPI)

fig_2_axs[0].set_title("Grayscale image")
fig_2_axs[0].imshow(steps.grayscale, cmap="gray")
fig_2_axs[0].set_axis_off()

fig_2_axs[1].set_title("Binary image")
fig_2_axs[1].imshow(steps.binary, cmap="binary")
fig_2_axs[1].set_axis_off()

fig_2_axs[2].set_title("Morphological dilated image")
fig_2_axs[2].imshow(steps.dilation, cmap="binary")
fig_2_axs[2].set_axis_off()

fig_step_2.savefig(os.path.join(GEN_PATH,"step#2.png"),transparent=True, frameon=False)

### Third Step: Watershed Segmentation

In [ ]:

fig_step_3, fig_3_axs = plt.subplots(ncols=3,figsize=(15,5), dpi=EXPORT_DPI)
colmap = mpl.cm.get_cmap("tab20b",np.unique(steps.labels).size) 

fig_3_axs[0].imshow(steps.distance, cmap="gray")
fig_3_axs[0].set_title("Distance map")
fig_3_axs[0].set_axis_off()

fig_3_axs[1].imshow(steps.labels,cmap=colmap)
fig_3_axs[1].set_title("Watershed segments")
fig_3_axs[1].set_axis_off()

imaxes=fig_3_axs[2].imshow(steps.filtered_labels,cmap=colmap)
fig_3_axs[2].set_title("Watershed filtered segments")
fig_3_axs[2].set_axis_off()
cbar = fig_step_3.colorbar(imaxes, ax=fig_3_axs[1:3], orientation="horizontal")
cbar.ax.set_title("Label id")
cbar.ax
fig_step_3.savefig(os.path.join(GEN_PATH,"step#3.png"),transparent=True, frameon=False)

### Detected cells

In [ ]:
fig_extracted, extracted_ax = plt.subplots(figsize=(10,10),dpi=300)

extracted_ax.imshow(steps.input)
pltutils.add_regions_annotations(steps.filtered_labels, extracted_ax, pltutils.box_annotator, fill=False, color="white",linewidth=1.25, linestyle="--")
pltutils.add_regions_annotations(steps.filtered_labels, extracted_ax, pltutils.label_annotator, color="white")